<a href="https://colab.research.google.com/github/wenxuan0923/My-notes/blob/master/style_pandas_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pandas Dataframe Styling

In this note I will apply **conditional formatting**, the visual styling of a DataFrame depending on the data within, by using the `DataFrame.style property`. The following techniques will be covered here:

- `Styler.applymap(func)`, which operates on DataFrames elementwise

- `Styler.apply(func, axis)`, which operates on DataFrames' column/rows
> Styler.apply(func, axis=0) for columnwise styles
>
> Styler.apply(func, axis=1) for rowwise styles
>
> Styler.apply(func, axis=None) for tablewise styles

- `Styler.background_gradient` create heatmap with dataframe data

- `Styler.bar`, include barchart in the dataframe

- `Styler.set_properties` apply styles doesn’t depend on values

- `Styler.set_table_styles` apply styles to the table as a whole

Reference: https://pandas.pydata.org/pandas-docs/version/0.18/style.html

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt

Dataset used in this note can be downloaded here: <a target='_blank' href='https://www.kaggle.com/kaggle/sf-salaries'>SF Salaries</a>.

In [42]:
df = pd.read_csv('SF_Salaries.csv')
# randomly pick 20 samples for simplicity
df = df.sample(20)
df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
47607,47608,Brenda Mitchell,Supervising Parts Storekeeper,72878,22347,221.3,34278.1,95446.26,129724.33,2012,NaN,San Francisco,NaN
70468,70469,Rickey White,IS Programmer Analyst,3730.33,0,0,1106.84,3730.33,4837.17,2012,NaN,San Francisco,NaN
53970,53971,Magdalena Zaldana,Patient Care Assistant,63035,4686.81,5090.9,30875,72812.71,103687.70,2012,NaN,San Francisco,NaN
45623,45624,Zoila Lechuga,Public SafetyComm Disp,79039.9,19331.2,5553.26,36750.9,103924.30,140675.23,2012,NaN,San Francisco,NaN
12824,12825,NORA ZAPATA-KREY,SENIOR PSYCHIATRIC SOCIAL WORKER,83522.2,0,2104,NaN,85626.23,85626.23,2011,NaN,San Francisco,NaN
121421,121422,Vernon C Abrams,Transit Operator,67135.4,24308.8,6671.97,36669.6,98116.22,134785.81,2014,NaN,San Francisco,FT
112759,112760,Tisileli T Lupeheke,Wire Rope Cable Maint Mechanic,81616.4,83894.5,8142.79,35269.3,173653.64,208922.90,2014,NaN,San Francisco,FT
88223,88224,Christine Gerber,Librarian 1,81964.3,0,1733.9,30226.8,83698.22,113924.99,2013,NaN,San Francisco,NaN
67955,67956,Marvin Cullado,Custodian,8612.7,0,623.1,5345.29,9235.80,14581.09,2012,NaN,San Francisco,NaN
140910,140911,Pastora A Ancheta,Special Nurse,25792.45,0.00,230.40,260.22,26022.85,26283.07,2014,NaN,San Francisco,PT


In [70]:
# Convert the monetary fields to numerical value
monetary_fields = ['BasePay', 'OvertimePay', 'OtherPay', 
                   'Benefits', 'TotalPay', 'TotalPayBenefits']
df[monetary_fields] = df[monetary_fields].apply(pd.to_numeric)

There are two types of methods when applying style to the dataframe:

- `Styler.applymap`: elementwise style

- `Styler.apply`: column-/row-/table-wise style

### Apply different colors base on a threshold with `Styler.applymap`
Make the text green for the cells whose `TotalPay` higher than average and red for value lower than average salary.

In [113]:
def color_lower_red(val):
    """
    'green' when TotalPay higher than average
    'red' when TotalPay lower than average
    """
    average_salary = 56516
    color = 'red' if val < average_salary else 'green'
    return 'color: %s' % color

df.style\
  .applymap(color_lower_red, subset=['TotalPay'])\
  .format({field:"${:20,.0f}" for field in monetary_fields}, na_rep="-")

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
47607,47608,Brenda Mitchell,Supervising Parts Storekeeper,"$ 72,878","$ 22,347",$ 221,"$ 34,278","$ 95,446","$ 129,724",2012,nan,San Francisco,nan
70468,70469,Rickey White,IS Programmer Analyst,"$ 3,730",$ 0,$ 0,"$ 1,107","$ 3,730","$ 4,837",2012,nan,San Francisco,nan
53970,53971,Magdalena Zaldana,Patient Care Assistant,"$ 63,035","$ 4,687","$ 5,091","$ 30,875","$ 72,813","$ 103,688",2012,nan,San Francisco,nan
45623,45624,Zoila Lechuga,Public SafetyComm Disp,"$ 79,040","$ 19,331","$ 5,553","$ 36,751","$ 103,924","$ 140,675",2012,nan,San Francisco,nan
12824,12825,NORA ZAPATA-KREY,SENIOR PSYCHIATRIC SOCIAL WORKER,"$ 83,522",$ 0,"$ 2,104",-,"$ 85,626","$ 85,626",2011,nan,San Francisco,nan
121421,121422,Vernon C Abrams,Transit Operator,"$ 67,135","$ 24,309","$ 6,672","$ 36,670","$ 98,116","$ 134,786",2014,nan,San Francisco,FT
112759,112760,Tisileli T Lupeheke,Wire Rope Cable Maint Mechanic,"$ 81,616","$ 83,894","$ 8,143","$ 35,269","$ 173,654","$ 208,923",2014,nan,San Francisco,FT
88223,88224,Christine Gerber,Librarian 1,"$ 81,964",$ 0,"$ 1,734","$ 30,227","$ 83,698","$ 113,925",2013,nan,San Francisco,nan
67955,67956,Marvin Cullado,Custodian,"$ 8,613",$ 0,$ 623,"$ 5,345","$ 9,236","$ 14,581",2012,nan,San Francisco,nan
140910,140911,Pastora A Ancheta,Special Nurse,"$ 25,792",$ 0,$ 230,$ 260,"$ 26,023","$ 26,283",2014,nan,San Francisco,PT


### Highlight the maximum value in the specified column with `Styler.apply`

In [119]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: rgba(237, 52, 52, 0.5)' if v else '' for v in is_max]

df.style\
  .apply(highlight_max, subset=monetary_fields)\
  .format({field:"${:20,.0f}" for field in monetary_fields}, na_rep="-")

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
47607,47608,Brenda Mitchell,Supervising Parts Storekeeper,"$ 72,878","$ 22,347",$ 221,"$ 34,278","$ 95,446","$ 129,724",2012,nan,San Francisco,nan
70468,70469,Rickey White,IS Programmer Analyst,"$ 3,730",$ 0,$ 0,"$ 1,107","$ 3,730","$ 4,837",2012,nan,San Francisco,nan
53970,53971,Magdalena Zaldana,Patient Care Assistant,"$ 63,035","$ 4,687","$ 5,091","$ 30,875","$ 72,813","$ 103,688",2012,nan,San Francisco,nan
45623,45624,Zoila Lechuga,Public SafetyComm Disp,"$ 79,040","$ 19,331","$ 5,553","$ 36,751","$ 103,924","$ 140,675",2012,nan,San Francisco,nan
12824,12825,NORA ZAPATA-KREY,SENIOR PSYCHIATRIC SOCIAL WORKER,"$ 83,522",$ 0,"$ 2,104",-,"$ 85,626","$ 85,626",2011,nan,San Francisco,nan
121421,121422,Vernon C Abrams,Transit Operator,"$ 67,135","$ 24,309","$ 6,672","$ 36,670","$ 98,116","$ 134,786",2014,nan,San Francisco,FT
112759,112760,Tisileli T Lupeheke,Wire Rope Cable Maint Mechanic,"$ 81,616","$ 83,894","$ 8,143","$ 35,269","$ 173,654","$ 208,923",2014,nan,San Francisco,FT
88223,88224,Christine Gerber,Librarian 1,"$ 81,964",$ 0,"$ 1,734","$ 30,227","$ 83,698","$ 113,925",2013,nan,San Francisco,nan
67955,67956,Marvin Cullado,Custodian,"$ 8,613",$ 0,$ 623,"$ 5,345","$ 9,236","$ 14,581",2012,nan,San Francisco,nan
140910,140911,Pastora A Ancheta,Special Nurse,"$ 25,792",$ 0,$ 230,$ 260,"$ 26,023","$ 26,283",2014,nan,San Francisco,PT


### Create "heatmaps" with the `background_gradient`

`Styler.background_gradient` takes the keyword arguments `low` and `high`. Roughly speaking these extend the range of your data by low and high percent so that when we convert the colors, the colormap's entire range isn't used, such that the color won't be too dark/light, we can still read the text. 

In [110]:
cmap = sns.cubehelix_palette(light=1, as_cmap=True)

df.style\
  .background_gradient(cmap=cmap, low=.2, high=0.8,
                       subset=monetary_fields)\
  .format("${:20,.0f}", na_rep="-",
          subset=monetary_fields)


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
47607,47608,Brenda Mitchell,Supervising Parts Storekeeper,"$ 72,878","$ 22,347",$ 221,"$ 34,278","$ 95,446","$ 129,724",2012,nan,San Francisco,nan
70468,70469,Rickey White,IS Programmer Analyst,"$ 3,730",$ 0,$ 0,"$ 1,107","$ 3,730","$ 4,837",2012,nan,San Francisco,nan
53970,53971,Magdalena Zaldana,Patient Care Assistant,"$ 63,035","$ 4,687","$ 5,091","$ 30,875","$ 72,813","$ 103,688",2012,nan,San Francisco,nan
45623,45624,Zoila Lechuga,Public SafetyComm Disp,"$ 79,040","$ 19,331","$ 5,553","$ 36,751","$ 103,924","$ 140,675",2012,nan,San Francisco,nan
12824,12825,NORA ZAPATA-KREY,SENIOR PSYCHIATRIC SOCIAL WORKER,"$ 83,522",$ 0,"$ 2,104",-,"$ 85,626","$ 85,626",2011,nan,San Francisco,nan
121421,121422,Vernon C Abrams,Transit Operator,"$ 67,135","$ 24,309","$ 6,672","$ 36,670","$ 98,116","$ 134,786",2014,nan,San Francisco,FT
112759,112760,Tisileli T Lupeheke,Wire Rope Cable Maint Mechanic,"$ 81,616","$ 83,894","$ 8,143","$ 35,269","$ 173,654","$ 208,923",2014,nan,San Francisco,FT
88223,88224,Christine Gerber,Librarian 1,"$ 81,964",$ 0,"$ 1,734","$ 30,227","$ 83,698","$ 113,925",2013,nan,San Francisco,nan
67955,67956,Marvin Cullado,Custodian,"$ 8,613",$ 0,$ 623,"$ 5,345","$ 9,236","$ 14,581",2012,nan,San Francisco,nan
140910,140911,Pastora A Ancheta,Special Nurse,"$ 25,792",$ 0,$ 230,$ 260,"$ 26,023","$ 26,283",2014,nan,San Francisco,PT


### Include Bar Charts in the dataframe

In [96]:
df[monetary_fields].style\
                   .set_caption('Include Bar charts in dataframe')\
                   .bar(subset=monetary_fields, 
                        align='mid', color='lightsteelblue') \
                   .highlight_null(null_color='seashell') \
                   .format("${:20,.0f}", na_rep="-")

,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits
47607,"$ 72,878","$ 22,347",$ 221,"$ 34,278","$ 95,446","$ 129,724"
70468,"$ 3,730",$ 0,$ 0,"$ 1,107","$ 3,730","$ 4,837"
53970,"$ 63,035","$ 4,687","$ 5,091","$ 30,875","$ 72,813","$ 103,688"
45623,"$ 79,040","$ 19,331","$ 5,553","$ 36,751","$ 103,924","$ 140,675"
12824,"$ 83,522",$ 0,"$ 2,104",-,"$ 85,626","$ 85,626"
121421,"$ 67,135","$ 24,309","$ 6,672","$ 36,670","$ 98,116","$ 134,786"
112759,"$ 81,616","$ 83,894","$ 8,143","$ 35,269","$ 173,654","$ 208,923"
88223,"$ 81,964",$ 0,"$ 1,734","$ 30,227","$ 83,698","$ 113,925"
67955,"$ 8,613",$ 0,$ 623,"$ 5,345","$ 9,236","$ 14,581"
140910,"$ 25,792",$ 0,$ 230,$ 260,"$ 26,023","$ 26,283"


### Apply style doesn’t actually depend on values with `Styler.set_properties`

In [131]:
df.style.set_properties(**{'background-color': 'steelblue',
                           'color': 'white',
                           'text-align': 'center',
                           'padding': '0.5rem',
                           'border-color': 'white'})

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
47607,47608,Brenda Mitchell,Supervising Parts Storekeeper,72878.000000,22346.960000,221.300000,34278.070000,95446.260000,129724.330000,2012,nan,San Francisco,nan
70468,70469,Rickey White,IS Programmer Analyst,3730.330000,0.000000,0.000000,1106.840000,3730.330000,4837.170000,2012,nan,San Francisco,nan
53970,53971,Magdalena Zaldana,Patient Care Assistant,63035.000000,4686.810000,5090.900000,30874.990000,72812.710000,103687.700000,2012,nan,San Francisco,nan
45623,45624,Zoila Lechuga,Public SafetyComm Disp,79039.880000,19331.160000,5553.260000,36750.930000,103924.300000,140675.230000,2012,nan,San Francisco,nan
12824,12825,NORA ZAPATA-KREY,SENIOR PSYCHIATRIC SOCIAL WORKER,83522.230000,0.000000,2104.000000,nan,85626.230000,85626.230000,2011,nan,San Francisco,nan
121421,121422,Vernon C Abrams,Transit Operator,67135.420000,24308.830000,6671.970000,36669.590000,98116.220000,134785.810000,2014,nan,San Francisco,FT
112759,112760,Tisileli T Lupeheke,Wire Rope Cable Maint Mechanic,81616.360000,83894.490000,8142.790000,35269.260000,173653.640000,208922.900000,2014,nan,San Francisco,FT
88223,88224,Christine Gerber,Librarian 1,81964.320000,0.000000,1733.900000,30226.770000,83698.220000,113924.990000,2013,nan,San Francisco,nan
67955,67956,Marvin Cullado,Custodian,8612.700000,0.000000,623.100000,5345.290000,9235.800000,14581.090000,2012,nan,San Francisco,nan
140910,140911,Pastora A Ancheta,Special Nurse,25792.450000,0.000000,230.400000,260.220000,26022.850000,26283.070000,2014,nan,San Francisco,PT


### Table Styles with `Styler.set_table_styles`

You need have some basic knowledge about css styling to apply table styles. The css style is passed in to the `props` argument with format `('style_name', 'value')`.

In [134]:
from IPython.display import HTML

def hover(hover_color="rgba(177, 201, 227, 0.4)"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
    
styles = [
    hover(),
    dict(selector=".col_heading", 
         props=[("font-size", "1.05rem"),
                ("text-align", "center")]),
    dict(selector="caption", 
         props=[("font-size", "1.3rem"),
                ("color", "steelblue"),
                ("padding", "1.2rem")])
]

html = (df.style
        .set_caption("Hover to highlight")
        .apply(highlight_max, subset=monetary_fields)
        .applymap(color_lower_red, subset=['TotalPay'])
        .bar(subset=['TotalPayBenefits'], 
             color='lightsteelblue') \
        .format({field:"${:20,.0f}" for field in monetary_fields}, na_rep="-")
        .set_properties(**{'padding': '0.3rem',
                           'border-color': 'white'})
        .set_table_styles(styles)
       )
html

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
47607,47608,Brenda Mitchell,Supervising Parts Storekeeper,"$ 72,878","$ 22,347",$ 221,"$ 34,278","$ 95,446","$ 129,724",2012,nan,San Francisco,nan
70468,70469,Rickey White,IS Programmer Analyst,"$ 3,730",$ 0,$ 0,"$ 1,107","$ 3,730","$ 4,837",2012,nan,San Francisco,nan
53970,53971,Magdalena Zaldana,Patient Care Assistant,"$ 63,035","$ 4,687","$ 5,091","$ 30,875","$ 72,813","$ 103,688",2012,nan,San Francisco,nan
45623,45624,Zoila Lechuga,Public SafetyComm Disp,"$ 79,040","$ 19,331","$ 5,553","$ 36,751","$ 103,924","$ 140,675",2012,nan,San Francisco,nan
12824,12825,NORA ZAPATA-KREY,SENIOR PSYCHIATRIC SOCIAL WORKER,"$ 83,522",$ 0,"$ 2,104",-,"$ 85,626","$ 85,626",2011,nan,San Francisco,nan
121421,121422,Vernon C Abrams,Transit Operator,"$ 67,135","$ 24,309","$ 6,672","$ 36,670","$ 98,116","$ 134,786",2014,nan,San Francisco,FT
112759,112760,Tisileli T Lupeheke,Wire Rope Cable Maint Mechanic,"$ 81,616","$ 83,894","$ 8,143","$ 35,269","$ 173,654","$ 208,923",2014,nan,San Francisco,FT
88223,88224,Christine Gerber,Librarian 1,"$ 81,964",$ 0,"$ 1,734","$ 30,227","$ 83,698","$ 113,925",2013,nan,San Francisco,nan
67955,67956,Marvin Cullado,Custodian,"$ 8,613",$ 0,$ 623,"$ 5,345","$ 9,236","$ 14,581",2012,nan,San Francisco,nan
140910,140911,Pastora A Ancheta,Special Nurse,"$ 25,792",$ 0,$ 230,$ 260,"$ 26,023","$ 26,283",2014,nan,San Francisco,PT
